OLA : 

Problem_statement : 

Whether drivers leave the company dues to various factors like age, gender, joining date and salary etc and try to predict the churn.

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('ola_driver_scaler.csv')

In [3]:
df.head(12)

,Unnamed: 0,MMM-YY,Driver_ID,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating
0,0,01/01/19,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,2381060,2
1,1,02/01/19,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,-665480,2
2,2,03/01/19,1,28.0,0.0,C23,2,57387,24/12/18,03/11/19,1,1,0,2
3,3,11/01/20,2,31.0,0.0,C7,2,67016,11/06/20,NaN,2,2,0,1
4,4,12/01/20,2,31.0,0.0,C7,2,67016,11/06/20,NaN,2,2,0,1
5,5,12/01/19,4,43.0,0.0,C13,2,65603,12/07/19,NaN,2,2,0,1
6,6,01/01/20,4,43.0,0.0,C13,2,65603,12/07/19,NaN,2,2,0,1
7,7,02/01/20,4,43.0,0.0,C13,2,65603,12/07/19,NaN,2,2,0,1
8,8,03/01/20,4,43.0,0.0,C13,2,65603,12/07/19,NaN,2,2,350000,1
9,9,04/01/20,4,43.0,0.0,C13,2,65603,12/07/19,27/04/20,2,2,0,1


Need to somehow bring the data each driver level

In [4]:
df.drop(columns='Unnamed: 0',inplace=True)

In [5]:
df.shape

(19104, 13)

In [9]:
df.isna().sum() / len(df)  * 100

MMM-YY                   0.000000
Driver_ID                0.000000
Age                      0.319305
Gender                   0.272194
City                     0.000000
Education_Level          0.000000
Income                   0.000000
Dateofjoining            0.000000
LastWorkingDate         91.541039
Joining Designation      0.000000
Grade                    0.000000
Total Business Value     0.000000
Quarterly Rating         0.000000
dtype: float64

Assumption : if the last working date is NaN that could possibly mean that the drivers stopped reporting to OLA or quit or changed the company.

In [7]:
df.duplicated().sum()

0

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19104 entries, 0 to 19103
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   MMM-YY                19104 non-null  object 
 1   Driver_ID             19104 non-null  int64  
 2   Age                   19043 non-null  float64
 3   Gender                19052 non-null  float64
 4   City                  19104 non-null  object 
 5   Education_Level       19104 non-null  int64  
 6   Income                19104 non-null  int64  
 7   Dateofjoining         19104 non-null  object 
 8   LastWorkingDate       1616 non-null   object 
 9   Joining Designation   19104 non-null  int64  
 10  Grade                 19104 non-null  int64  
 11  Total Business Value  19104 non-null  int64  
 12  Quarterly Rating      19104 non-null  int64  
dtypes: float64(2), int64(7), object(4)
memory usage: 1.9+ MB


In [11]:
for each in  ["MMM-YY","Dateofjoining","LastWorkingDate"] : 
    df[each] = pd.to_datetime(df[each])

Missing value imputation : 

Gender : Gender of the driver – Male : 0, Female: 1

In [27]:
# age , gender , last working date 

df["Gender"].value_counts(dropna=False)

0.0    11074
1.0     7978
NaN       52
Name: Gender, dtype: int64

Education_Level : Education level – 0 for 10+ ,1 for 12+ ,2 for graduate

In [28]:
df["Education_Level"].value_counts()

1    6864
2    6327
0    5913
Name: Education_Level, dtype: int64

Let's try Knn imputation -  for this we need to consider the numerical features 

In [30]:
num_df = df.select_dtypes(np.number)

In [32]:
num_df.columns

Index(['Driver_ID', 'Age', 'Gender', 'Education_Level', 'Income',
       'Joining Designation', 'Grade', 'Total Business Value',
       'Quarterly Rating'],
      dtype='object')

In [33]:
num_df.drop(columns='Driver_ID',inplace=True)

In [35]:
# knn imputation 

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean',)

df[num_df.columns] = imputer.fit_transform(df[num_df.columns])

nan_euclidean trivia :

The nan_euclidean metric is a special metric provided by scikit-learn's PairwiseDistances class specifically for handling missing values (NaNs) in the data. It is similar to the Euclidean distance metric, but it treats NaNs as a separate category and assigns a large distance to any pair of samples that contains at least one NaN.

This is useful when you have missing values in your data and you want to impute them using the KNNImputer. By using the nan_euclidean metric, the KNNImputer will ignore the NaNs when finding the nearest neighbors, ensuring that the imputed values are based on the non-missing data.

In [36]:
df.isna().sum() / len(df)  * 100

MMM-YY                   0.000000
Driver_ID                0.000000
Age                      0.000000
Gender                   0.000000
City                     0.000000
Education_Level          0.000000
Income                   0.000000
Dateofjoining            0.000000
LastWorkingDate         91.541039
Joining Designation      0.000000
Grade                    0.000000
Total Business Value     0.000000
Quarterly Rating         0.000000
dtype: float64

let's bring down the data to the each driver level :

In [37]:
df[df["Driver_ID"] == 500]

,MMM-YY,Driver_ID,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating
3342,2019-01-01,500,39.0,0.0,C12,1.0,103119.0,2018-05-11,NaT,4.0,4.0,612140.0,1.0
3343,2019-02-01,500,39.0,0.0,C12,1.0,103119.0,2018-05-11,NaT,4.0,4.0,508740.0,1.0
3344,2019-03-01,500,39.0,0.0,C12,1.0,103119.0,2018-05-11,NaT,4.0,4.0,1683170.0,1.0
3345,2019-04-01,500,39.0,0.0,C12,1.0,103119.0,2018-05-11,NaT,4.0,4.0,402470.0,1.0
3346,2019-05-01,500,39.0,0.0,C12,1.0,103119.0,2018-05-11,NaT,4.0,4.0,0.0,1.0
3347,2019-06-01,500,39.0,0.0,C12,1.0,103119.0,2018-05-11,NaT,4.0,4.0,0.0,1.0
3348,2019-07-01,500,40.0,0.0,C12,1.0,103119.0,2018-05-11,NaT,4.0,4.0,300000.0,3.0
3349,2019-08-01,500,40.0,0.0,C12,1.0,103119.0,2018-05-11,NaT,4.0,4.0,2654880.0,3.0
3350,2019-09-01,500,40.0,0.0,C12,1.0,103119.0,2018-05-11,NaT,4.0,4.0,1435040.0,3.0
3351,2019-10-01,500,40.0,0.0,C12,1.0,103119.0,2018-05-11,2019-10-10,4.0,4.0,0.0,1.0


The data is at montly level and we need to bring it to the driver level and if we observe each driver reports at OLA driver hub at the start of the month , first let's do a groupby at the driver level, then monthly level

In [40]:
# age : max , gender : first , city : first  , education : last , income :  last , dateofjoining : last , 
# lastworkingdate : last , joining designation : last , grade : last , total business value : sum , quarterly rating : last

agg_dict = {'Age':'max', 'Gender':'first','City':'first',
             'Education_Level':'last','Income':'last',
             'Dateofjoining':'last','LastWorkingDate':'last',
             'Joining Designation':'last','Grade':'last',
             'Total Business Value':'sum','Quarterly Rating':'last'}

df1 = df.groupby(['Driver_ID','MMM-YY']).aggregate(agg_dict)


Now the data is at driver level for each month 

In [41]:
df1

Age  Gender City  Education_Level   Income  \
Driver_ID MMM-YY                                                    
1         2019-01-01  28.0     0.0  C23              2.0  57387.0   
          2019-02-01  28.0     0.0  C23              2.0  57387.0   
          2019-03-01  28.0     0.0  C23              2.0  57387.0   
2         2020-11-01  31.0     0.0   C7              2.0  67016.0   
          2020-12-01  31.0     0.0   C7              2.0  67016.0   
...                    ...     ...  ...              ...      ...   
2788      2020-08-01  30.0     0.0  C27              2.0  70254.0   
          2020-09-01  30.0     0.0  C27              2.0  70254.0   
          2020-10-01  30.0     0.0  C27              2.0  70254.0   
          2020-11-01  30.0     0.0  C27              2.0  70254.0   
          2020-12-01  30.0     0.0  C27              2.0  70254.0   

                     Dateofjoining LastWorkingDate  Joining Designation  \
Driver_ID MMM-YY                                                          
1         2019-01-01    2018-12-24             NaT                  1.0   
          2019-02-01    2018-12-24             NaT                  1.0   
          2019-03-01    2018-12-24      2019-03-11                  1.0   
2         2020-11-01    2020-11-06             NaT                  2.0   
          2020-12-01    2020-11-06             NaT                  2.0   
...                            ...             ...                  ...   
2788      2020-08-01    2020-06-08             NaT                  2.0   
          2020-09-01    2020-06-08             NaT                  2.0   
          2020-10-01    2020-06-08             NaT                  2.0   
          2020-11-01    2020-06-08             NaT                  2.0   
          2020-12-01    2020-06-08             NaT                  2.0   

                      Grade  Total Business Value  Quarterly Rating  
Driver_ID MMM-YY                                                     
1         2019-01-01    1.0             2381060.0               2.0  
          2019-02-01    1.0             -665480.0               2.0  
          2019-03-01    1.0                   0.0               2.0  
2         2020-11-01    2.0                   0.0               1.0  
          2020-12-01    2.0                   0.0               1.0  
...                     ...                   ...               ...  
2788      2020-08-01    2.0              740280.0               3.0  
          2020-09-01    2.0              448370.0               3.0  
          2020-10-01    2.0                   0.0               2.0  
          2020-11-01    2.0              200420.0               2.0  
          2020-12-01    2.0              411480.0               2.0  

[19104 rows x 11 columns]

In [45]:
df1.reset_index(inplace=True)

In [47]:
df1.sort_values(by=['Driver_ID','MMM-YY'],inplace=True)

In [48]:
df1

,Driver_ID,MMM-YY,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating
0,1,2019-01-01,28.0,0.0,C23,2.0,57387.0,2018-12-24,NaT,1.0,1.0,2381060.0,2.0
1,1,2019-02-01,28.0,0.0,C23,2.0,57387.0,2018-12-24,NaT,1.0,1.0,-665480.0,2.0
2,1,2019-03-01,28.0,0.0,C23,2.0,57387.0,2018-12-24,2019-03-11,1.0,1.0,0.0,2.0
3,2,2020-11-01,31.0,0.0,C7,2.0,67016.0,2020-11-06,NaT,2.0,2.0,0.0,1.0
4,2,2020-12-01,31.0,0.0,C7,2.0,67016.0,2020-11-06,NaT,2.0,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19099,2788,2020-08-01,30.0,0.0,C27,2.0,70254.0,2020-06-08,NaT,2.0,2.0,740280.0,3.0
19100,2788,2020-09-01,30.0,0.0,C27,2.0,70254.0,2020-06-08,NaT,2.0,2.0,448370.0,3.0
19101,2788,2020-10-01,30.0,0.0,C27,2.0,70254.0,2020-06-08,NaT,2.0,2.0,0.0,2.0
19102,2788,2020-11-01,30.0,0.0,C27,2.0,70254.0,2020-06-08,NaT,2.0,2.0,200420.0,2.0


In [49]:
# bringing the data to the driver level 
#  age : max , gender : last, city : last  , education : last , income :  last ,   
# lastworkingdate : last , joining designation : last , grade : last , total business value : sum , quarterly rating : last

add_driver_dict = {'Age':'max', 'Gender':'last','City':'last',
             'Education_Level':'last','Income':'last',
             'Dateofjoining':'last','LastWorkingDate':'last',
             'Joining Designation':'last','Grade':'last',
             'Total Business Value':'sum','Quarterly Rating':'last'}

df_driver = df1.groupby('Driver_ID').aggregate(add_driver_dict)

In [52]:
df1["Driver_ID"].nunique()

2381

In [50]:
df_driver

,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating
Driver_ID,,,,,,,,,,,
1,28.0,0.0,C23,2.0,57387.0,2018-12-24,2019-03-11,1.0,1.0,1715580.0,2.0
2,31.0,0.0,C7,2.0,67016.0,2020-11-06,NaT,2.0,2.0,0.0,1.0
4,43.0,0.0,C13,2.0,65603.0,2019-12-07,2020-04-27,2.0,2.0,350000.0,1.0
5,29.0,0.0,C9,0.0,46368.0,2019-01-09,2019-03-07,1.0,1.0,120360.0,1.0
6,31.0,1.0,C11,1.0,78728.0,2020-07-31,NaT,3.0,3.0,1265000.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
2784,34.0,0.0,C24,0.0,82815.0,2015-10-15,NaT,2.0,3.0,21748820.0,4.0
2785,34.0,1.0,C9,0.0,12105.0,2020-08-28,2020-10-28,1.0,1.0,0.0,1.0
2786,45.0,0.0,C19,0.0,35370.0,2018-07-31,2019-09-22,2.0,2.0,2815090.0,1.0


In [69]:
df_driver.reset_index(inplace=True)

In [70]:
df_driver.head(4)

,Driver_ID,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating
0,1,28.0,0.0,C23,2.0,57387.0,2018-12-24,2019-03-11,1.0,1.0,1715580.0,2.0
1,2,31.0,0.0,C7,2.0,67016.0,2020-11-06,NaT,2.0,2.0,0.0,1.0
2,4,43.0,0.0,C13,2.0,65603.0,2019-12-07,2020-04-27,2.0,2.0,350000.0,1.0
3,5,29.0,0.0,C9,0.0,46368.0,2019-01-09,2019-03-07,1.0,1.0,120360.0,1.0


Feature Engineering :

Creating a column which tells whether the quarterly rating has increased for that driver - for those whose quarterly rating has increased we assign the value 1

In [58]:
# difference between quareterly rating min and max 

quarterly_rating_change = df1.groupby('Driver_ID').aggregate({'Quarterly Rating':['first','last']}).reset_index()


In [61]:
quarterly_rating_change.columns = ['Driver_ID','Quarterly Rating_first','Quarterly Rating_last']
quarterly_rating_change["quarterly_rating_inc"] = quarterly_rating_change["Quarterly Rating_last"] > quarterly_rating_change["Quarterly Rating_first"]
quarterly_rating_change["quarterly_rating_inc"] = np.where(quarterly_rating_change["quarterly_rating_inc"]==True,1,0)

In [72]:
quarterly_rating_change.head(2)

,Driver_ID,Quarterly Rating_first,Quarterly Rating_last,quarterly_rating_inc
0,1,2.0,2.0,0
1,2,1.0,1.0,0


In [73]:
# add quarterly_rating_inc to df_driver 

df_driver = pd.merge(df_driver,quarterly_rating_change[["Driver_ID","quarterly_rating_inc"]],how='inner',on='Driver_ID')

In [75]:
# df_driver["quarterly_rating_inc_1"] = quarterly_rating_change["quarterly_rating_inc"].values
# df_driver.drop(["quarterly_rating_inc_1"],axis=1,inplace=True)

Target variable creation: Creating a column called target which tells whether the driver has left the company- driver whose last working day is present will have the value 1

In [82]:
# if last working date is not null then change the value to 1 else 0

df_driver["last_working_date_target"] = np.where(df_driver["LastWorkingDate"].isnull(),0,1)

In [84]:
#df_driver.drop(["last_working_date_inc"],axis=1,inplace=True)

Creating a column which tells whether the monthly income has increased for that driver - for those whose monthly income has increased we assign the value 1

In [86]:
# monthly income increase 

monthly_inc = df1.groupby('Driver_ID').aggregate({'Income':['first','last']}).reset_index()

monthly_inc.columns = ['Driver_ID','Income_first','Income_last']
monthly_inc["monthly_income_increase"] = np.where(monthly_inc["Income_last"] > monthly_inc["Income_first"],1,0)
df_driver["monthly_income_increase"] = monthly_inc["monthly_income_increase"].values

In [93]:
df_driver.head(5)

,Driver_ID,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating,quarterly_rating_inc,last_working_date_target,monthly_income_increase
0,1,28.0,0.0,C23,2.0,57387.0,2018-12-24,2019-03-11,1.0,1.0,1715580.0,2.0,0,1,0
1,2,31.0,0.0,C7,2.0,67016.0,2020-11-06,NaT,2.0,2.0,0.0,1.0,0,0,0
2,4,43.0,0.0,C13,2.0,65603.0,2019-12-07,2020-04-27,2.0,2.0,350000.0,1.0,0,1,0
3,5,29.0,0.0,C9,0.0,46368.0,2019-01-09,2019-03-07,1.0,1.0,120360.0,1.0,0,1,0
4,6,31.0,1.0,C11,1.0,78728.0,2020-07-31,NaT,3.0,3.0,1265000.0,2.0,1,0,0


Why are calculating quarterly rating change and monthly income increase ? 

Because in real world driver rating directly affects the income and bonuses received. If the rating hasn't improved then the income will not increase. 
So, if the income hasn't increased then drivers potentially will be leaving the company.

So we need to calculate the change in rating and monthly income.

In [94]:
df_driver.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2381 entries, 0 to 2380
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Driver_ID                 2381 non-null   int64         
 1   Age                       2381 non-null   float64       
 2   Gender                    2381 non-null   float64       
 3   City                      2381 non-null   object        
 4   Education_Level           2381 non-null   float64       
 5   Income                    2381 non-null   float64       
 6   Dateofjoining             2381 non-null   datetime64[ns]
 7   LastWorkingDate           1616 non-null   datetime64[ns]
 8   Joining Designation       2381 non-null   float64       
 9   Grade                     2381 non-null   float64       
 10  Total Business Value      2381 non-null   float64       
 11  Quarterly Rating          2381 non-null   float64       
 12  quarterly_rating_inc

In [96]:
df_driver.drop(["LastWorkingDate"],axis=1,inplace=True)

In [99]:
df_driver.describe().T

,count,mean,std,min,25%,50%,75%,max
Driver_ID,2381.0,1.397559e+03,8.061616e+02,1.0,695.0,1400.0,2100.0,2788.0
Age,2381.0,3.377018e+01,5.933265e+00,21.0,30.0,33.0,37.0,58.0
Gender,2381.0,4.105838e-01,4.914963e-01,0.0,0.0,0.0,1.0,1.0
Education_Level,2381.0,1.007560e+00,8.162900e-01,0.0,0.0,1.0,2.0,2.0
Income,2381.0,5.933416e+04,2.838367e+04,10747.0,39104.0,55315.0,75986.0,188418.0
Joining Designation,2381.0,1.820244e+00,8.414334e-01,1.0,1.0,2.0,2.0,5.0
Grade,2381.0,2.096598e+00,9.415218e-01,1.0,1.0,2.0,3.0,5.0
Total Business Value,2381.0,4.586742e+06,9.127115e+06,-1385530.0,0.0,817680.0,4173650.0,95331060.0
Quarterly Rating,2381.0,1.427971e+00,8.098389e-01,1.0,1.0,1.0,2.0,4.0
quarterly_rating_inc,2381.0,1.503570e-01,3.574961e-01,0.0,0.0,0.0,0.0,1.0
